# IAB - Hands-on Tutorial for Link Analysis

Welcome to IAB - hands-on tutorial for link analysis. 
In this tutorial, we will study several techniques for link analysis in graphs. 
This tutorial consists of two sessions and one homework, and each of them will handle the following topic:

* **Session 1**. Tutorial on PageRank - Part 1 (60 mins)
* **Session 2**. Tutorial on PageRank - Part 2 (60 mins)
* **Session 3**. Tutorial on Topic-specific PageRank (120 mins)
* <span style="color:blue">**Homework**. Implementation of HITS</span>

We recommend fully understanding the lecture videos related to link analysis (or ranking) models such as PageRank, Topic-specific PageRank, and HITS before entering this tutorial since we will **NOT** explain the theoretical backgrounds on these techniques during the tutorial. 
We will mainly focus on how to implement the algorithms of those models and how to rank nodes in real-world graphs using those ranking models. 

The main contributors of this material are as follows:
* *Junghoon Kim* (rlawjdgns434@snu.ac.kr)
* *Minyong Cho* (minyongcho@snu.ac.kr)
* *U Kang* (ukang@snu.ac.kr)
------

-----
## Homework. Implementation of HITS
In this homework, you will implement HITS in Python. 
The main goals of this homework are summarized as follows:
* **Goal 1.** How to implement HITS based on sparse matrices using `numpy` and `scipy` in Python
* **Goal 2.** To perform a qualitative analysis of the ranking result from HITS in real-world networks

The outline of this homework is as follows:
* **Step 1.** Review HITS algorithm
* **Step 2.** Implement HITS algorithm - the sparse matrix version
* **Step 3.** Validate your HITS implementation
* **Step 4.** Qualitative analysis of the ranking result from HITS

<span style="color:blue">Note that we will evaluate the result from Step 2 as this homework. Before submittig your homework, please verify if your implementation is correct at Step 3. <span>

------

-----
### Step 1. Review HITS algorithm

In this step, we will briefly review HITS algorithm which is another global node ranking model in graphs. 

#### Step 1-1. Mathematical definition of HITS
Given an adjacency matrix of a graph, HITS computes the hub and authority score vectors which can be used for ranking nodes in the graph. 

##### Problem definition of HITS
* **Input**: adjacency matrix $\mathbf{A}\in \mathbb{R}^{n \times n}$ of a graph $G=(V, E)$
* **Output**: hub score vector $\mathbf{h} \in \mathbb{R}^{n}$, and authority score vector $\mathbf{a} \in \mathbb{R}^{n}$ such that

$$
\mathbf{h} = \mathbf{A}\mathbf{a} \\
\mathbf{a} = \mathbf{A}^{\top}\mathbf{h}
$$

* The above equations are called the equations for HITS.

Note that HITS does not require the row-normalized adjacency matrix. 

#### Step 1-2. Iterative Algorithm for HITS
As described in the lecture video, the hub and authority score vectors, $\mathbf{h}$ and $\mathbf{a}$, are obtained by iteratively computing the equations for HITS.   
Note that for each iteration, we need to L2-normalize the vectors $\mathbf{h}$ and $\mathbf{a}$.
The following pseudo-code represents the iterative algorithm for HITS. 

<img src="./images/iterative-algorithm-hits.png" width="400">

Implement the iterative algorithm in Python. 


------

### Step 2. Implement HITS algorithm - the sparse matrix version
Implement the sparse matrix version of HITS algorithm in Python.

In [1]:
try:
    import numpy
except:
    !pip install numpy
    import numpy
try:
    import scipy
except:
    !pip install scipy
    import scipy    
try:
    import matplotlib
except:
    !pip install matplotlib
    import matplotlib
try:
    import pandas
except:
    !pip install pandas
    import pandas
try:
    from IPython.display import display 
except:
    !pip install ipython
    from IPython.display import display 

#### Step 2-2. Implement the phase for loading the graph dataset
In this step, we will implement the phase for loading the graph dataset of the sparse matrix version of HITS. 
We first need to import the following packages for this.

In [2]:
# the below commands restrict the number of computation threads to 1
import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

import numpy as np
from scipy.sparse import csr_matrix, find
import pandas as pd
from IPython.display import display

As in the previous sessions, we implement the following function for loading graph datasets.
Note that we do not need to modify the function since the functionality in the version is the same with that of Session 2.

In [3]:
class SparseHITS:
    def load_graph_dataset(self, data_home, is_undirected=False):
        '''
        Load the graph dataset from the given directory (data_home)

        inputs:
            data_home: string
                directory path conatining a dataset (edges.tsv, node_labels.tsv)
            is_undirected: bool
                if the graph is undirected
        '''
        # Step 1. self file paths from data_home
        edge_path = "{}/edges.tsv".format(data_home)
        
        # Step 2. read the list of edges from edge_path
        edges = np.loadtxt(edge_path, dtype=int)
        n = int(np.amax(edges[:, 0:2])) + 1
        
        # Step 3. convert the edge list to the weighted adjacency matrix
        rows = edges[:, 0]
        cols = edges[:, 1]
        weights = edges[:, 2]
        self.A = csr_matrix((weights, (rows, cols)), shape=(n, n), dtype=float)
        if is_undirected == True:
            self.A = self.A + self.A.T
        self.AT = self.A.T
                
        # Step 4. set n (# of nodes) and m (# of edges)
        self.n = self.A.shape[0]     # number of nodes
        self.m = self.A.nnz          # number of edges

In [4]:
class SparseHITS(SparseHITS):
    def load_node_labels(self, data_home):
        '''
        Load the node labels from the given directory (data_home)

        inputs:
            data_home: string
                directory path conatining a dataset
        '''
        label_path = "{}/node_labels.tsv".format(data_home)
        self.node_labels = pd.read_csv(label_path, sep="\t")        

Let's check if the function is correctly implemented. 
We will use the small dataset at `./data/small` as before.

In [5]:
hits = SparseHITS()
hits.load_graph_dataset('./data/small', is_undirected=False)
print("The number n of nodes: {}".format(hits.n))
print("The number m of edges: {}".format(hits.m))

The number n of nodes: 29
The number m of edges: 376


#### Step 2-3. Implement the iterative phase (Homework)

Contrary to PageRank, HITS does not need a normalization phase for the adjacency matrix. 
In this step, we implement the iterative phase of HITS in the following algorithm.

<img src="./images/iterative-algorithm-hits.png" width="400">

In [6]:
class SparseHITS(SparseHITS):
    def iterate_HITS(self, epsilon=1e-9, maxIters=100):
        '''
        ///Homework!///
        Iterate the HITS equation to obatin the hub & authority score vectors
        
        inputs:
            epsilon: float
                the error tolerance of the iteration
            maxIters: int
                the maximum number of iterations
                
        outputs:
            h: np.ndarray (n x 1 vector)
                the final hub score vector
            a: np.ndarray (n x 1 vector)
                the final authority score vector
            h_residuals: list
                the list of hub residuals over the iteration
            a_residuals: list
                the list of authority residuals over the iteration

        '''        
        # From ==========================
        # implement the HITS algorithm
        # To ============================
        
        # since SVD is not unique, h and a could be negative according to a random seed
        # in this case, we need to make the scores non-negative and L2-normalize them out
        h = h * np.sign(h)
        h = h / np.linalg.norm(h, 2)
        a = a * np.sign(a)
        a = a / np.linalg.norm(a, 2)
        
        return h, a, h_residuals, a_residuals

Let's check the implementation is correctly performed. 
Please check ranking results based on hub and authority scores using `pandas`. 

**Homework. ** Print your top-$5$ ranking results on the files `hub.csv` and `auth.csv`, respectively.
* format: each line print node id and its hub or authority score where they are comma-separated, i.e., `node_id, score`. Each score should be down to $4$ places of decimals. 
* `hub.csv`: it should contain top-$5$ rankings sorted in the decreasing order of the hub scores. 
* `auth.csv`: it should contain top-$5$ rankings sorted in the decreasing order of the authority scores. 
* To write an object of `pandas` to a file, please try to use `to_csv` function of `pandas`. 
    - https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html

In [7]:
import pandas as pd

def rank_nodes(ranking_scores, topk=-1):
    sorted_nodes = np.flipud(np.argsort(ranking_scores))
    sorted_scores = ranking_scores[sorted_nodes]
    ranking_results = pd.DataFrame()
    ranking_results["node_id"] = sorted_nodes
    ranking_results["score"] = sorted_scores
    
    return ranking_results[0:topk]

In [8]:
hits = SparseHITS()
hits.load_graph_dataset('./data/small', is_undirected=False)
h, a, _, _ = hits.iterate_HITS(epsilon=1e-9, maxIters=100)

print("Top-5 rankings based on the hub score vector:")
display(rank_nodes(h, 5))

print("Top-5 rankings based on the authority score vector")
display(rank_nodes(a, 5))

# Homework!
# Print your results on files 'hub.csv' and 'auth.csv', respectively.
# Reference: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_csv.html
# rank_nodes(h,5).to_csv('hub.csv', sep=',', index=False, header=False, float_format='%.4f')
# rank_nodes(a,5).to_csv('auth.csv', sep=',', index=False, header=False, float_format='%.4f')

UnboundLocalError: local variable 'h' referenced before assignment

-----
### Step 3. Validate your HITS implementation

Validate the implementation of HITS.
Similarly to PageRank, check if the residuals of the iterative algorithm monotonically decrease, and compare between the exact and iterative solutions of HITS.

#### Step 3-1. Check if the residuals of the iterative algorithm monotonically decrease

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 13}) # set the font-size in the figure

def plot_residuals(residuals, title):
    plt.semilogy(residuals, marker='o', markersize=5)
    plt.title(title)
    plt.ylim(ymin=1e-10, ymax=1e1)
    plt.ylabel('Residual')
    plt.xlabel('# of iterations')
    plt.grid(True)
    plt.show()

Let's check the plot for the hub residuals first.

In [ ]:
hits = SparseHITS()
hits.load_graph_dataset('./data/small', is_undirected=False)
h, a, h_residuals, a_residuals = hits.iterate_HITS(epsilon=1e-9, maxIters=100)

plot_residuals(h_residuals, 'Change of Hub Residuals from HITS')

Next, the following will plot the authority residuals.

In [ ]:
plot_residuals(a_residuals, 'Change of Authority Residuals from HITS')

As you can see, those residuals monotonically decrease as the number of iteration increases.

#### Step 3-2. Check if the iterative solution is equal to the exact solution of HITS

As explained in the lecture video, we can directly compute the hub ans authority score vectors of HITS as follows:
* The hub score vector $\mathbf{h}$ is the principle eigenvector of $\mathbf{A}\mathbf{A}^{\top}$.
* The authority score vector $\mathbf{a}$ is the principle eigenvector of $\mathbf{A}^{\top}\mathbf{A}$.

The principle eigenvector of a matrix is an eigenvector of the largest eigenvalue. 
Suppose the Singular Value Decomposition (SVD) of the adjacency matrix is $\mathbf{A}=\mathbf{U}\mathbf{S}\mathbf{V}^{\top}$. Then,
* $\mathbf{h}$ is the first (most left) vector of $\mathbf{U}$. 
* $\mathbf{a}$ is the first (most left) vector of $\mathbf{V}$.

We will not describe the detailed proofs for the above statements, but we can use them as the exact solutions for the hub and authority score vectors. 

We will implement the following function to compute the exact solutions. 
Note that since the adjacency matrix $\mathbf{A}$ is a sparse matrix, we will use `svds` (sparse SVD) in `scipy`.
* `svds`: this performs SVD on a sparse matrix
    - https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.svds.html

In [ ]:
from scipy.sparse.linalg import svds

class SparseHITS(SparseHITS):
    def compute_exact_HITS(self):
        '''
        Compute the exact hub & authority score vectors from the closed form

        outputs:
            h: np.ndarray (n x 1 vector)
                the final hub score vector
            a: np.ndarray (n x 1 vector)
                the final authority score vector
        '''
        h, s, a = svds(self.A, k=1)
        
        h = np.asarray(h).flatten()   
        a = np.asarray(a).flatten()
        
        return h, a

Let's check the error between the exact and iterative solutions of HITS.

In [ ]:
hits = SparseHITS()
hits.load_graph_dataset('./data/small', is_undirected=False)
h_exact, a_exact = hits.compute_exact_HITS()
h_iter, a_iter, _, _ = hits.iterate_HITS(epsilon=1e-9, maxIters=100)

h_error = np.linalg.norm(h_exact - h_iter, 1)
print("Error between exact and iterative hub scores: {:e}".format(h_error))

a_error = np.linalg.norm(a_exact - a_iter, 1)
print("Error between exact and iterative authority scores: {:e}".format(a_error))

-----
### Step 4. Qualitative analysis of the ranking result from HITS

Now, we will do a qualitative analysis of the ranking result from HITS on a real-world graph. 
For this, we first use the following code which is already implemented through the previous session. 

In [ ]:
class SparseHITS(SparseHITS):
    def rank_nodes(self, ranking_scores, topk=-1):
        '''
        Rank nodes in the order of given ranking scores. 
        This function reports top-k rankings. 

        inputs:
            ranking_scores: np.ndarray
                ranking score vector
            topk: int
                top-k ranking parameter, default is -1 indicating report all ranks
        '''
        sorted_nodes = np.flipud(np.argsort(ranking_scores)) # argsort in the descending order
        sorted_scores = ranking_scores[sorted_nodes]         # sort the ranking scores
        ranks = range(1, self.n+1) # 0~n-1

        result_labels = self.node_labels.iloc[sorted_nodes][0:topk]
        result_labels.insert(0, "rank", ranks[0:topk])
        result_labels["score"] = sorted_scores[0:topk]
        result_labels.reset_index(drop = True, inplace = True)
        return result_labels

The dataset used in this step is *enron* at `./data/enron` which has been used in Session 2. 
Please refer to Session 2 for the detailed description for the dataset. 

After loading the dataset, we will compute the hub ans authority score vectors of HITS.

In [ ]:
data_home = './data/enron'
hits = SparseHITS()
hits.load_graph_dataset(data_home, is_undirected=False)
hits.load_node_labels(data_home)
h, a, h_residuals, a_residuals = hits.iterate_HITS(epsilon=1e-9, maxIters=100)

Let's check the top-10 rankings based on the hub score vector in the network.

In [ ]:
print("Top-10 rankings based on the hub score vector:")
display(hits.rank_nodes(h, topk=10))

Let's check the top-10 rankings based on the authority score vector in the network.

In [ ]:
print("Top-10 rankings based on the authority score vector:")
display(hits.rank_nodes(a, topk=10))

-----
## Homework. Summary

In this homework, you implemented HITS algorithm (the sparse matrix version) in Python. 
More specifically, you are able to answer the following goals now. 

* **Goal 1.** How to implement HITS based on sparse matrices using `numpy` and `scipy` in Python
    - You implemented the iterative algorithm for HITS based on sparse matrices.
* **Goal 2.** To perform a qualitative analysis of the ranking result from HITS in real-world networks
    - You performed a qualitative analysis on the `enron` dataset which is a real-world network.